In [1]:
%load_ext autoreload
%autoreload 2

# Examples of CORDEX-CMIP6 cmorization

This notebook should show some examples of how a cmorized CORDEX-CMIP6 dataset could look like.

In [2]:
!ls ../Tables

CMIP6_CV.json             CORDEX_6hr.json           CORDEX_fx.json
CMIP6_coordinate.json     CORDEX_CV.json            CORDEX_grids.json
CMIP6_formula_terms.json  CORDEX_coordinate.json    CORDEX_mon.json
CMIP6_grids.json          CORDEX_day.json           CORDEX_remo_example.json
CORDEX_1hr.json           CORDEX_formula_terms.json


## Controlled Vocabulary

The current `CORDEX-CMIP6_CV.json` file contains the controlled vocabulary used by the CMOR3 library for rewriting the model output. Let's have a look:

In [3]:
import json
import pprint

from IPython.display import JSON

# Opening JSON file
with open("../Tables/CORDEX_CV.json") as json_file:
    CV = json.load(json_file)["CV"]

# JSON(CV['required_global_attributes'])
pprint.pprint(CV["required_global_attributes"])

['Conventions',
 'activity_id',
 'contact',
 'creation_date',
 'domain_id',
 'domain',
 'driving_experiment',
 'driving_experiment_id',
 'driving_institution_id',
 'driving_source_id',
 'driving_variant_label',
 'frequency',
 'further_info_url',
 'institution',
 'institution_id',
 'mip_era',
 'native_resolution',
 'product',
 'project_id',
 'realm',
 'source',
 'source_id',
 'source_type',
 'version_realization',
 'tracking_id',
 'variable_id',
 'license']


## Cmorization example

We use the `pyremo.cmor` module to create an example of a CORDEX-CMIP6 dataset.

In [4]:
import os

import cordex as cx
import xarray as xr
from cordex import cmor as cmor

table_dir = "../Tables"

cmor.set_options(table_prefix="CORDEX")


def test_cmorizer_fx():
    ds = cx.cordex_domain("EUR-11", dummy="topo")
    filename = cmor.cmorize_variable(
        ds,
        "orog",
        mapping_table={"orog": {"varname": "topo"}},
        cmor_table=os.path.join(table_dir, "CORDEX_fx.json"),
        dataset_table=os.path.join(table_dir, "CORDEX_remo_example.json"),
        grids_table=os.path.join(table_dir, "CORDEX_grids.json"),
        CORDEX_domain="EUR-11",
        time_units=None,
        allow_units_convert=True,
    )
    return filename


def test_cmorizer_mon():
    ds = cx.tutorial.open_dataset("remo_EUR-11_TEMP2_mon")
    filename = cmor.cmorize_variable(
        ds,
        "tas",
        mapping_table={"tas": {"varname": "TEMP2"}},
        cmor_table=os.path.join(table_dir, "CORDEX_mon.json"),
        dataset_table=os.path.join(table_dir, "CORDEX_remo_example.json"),
        grids_table=os.path.join(table_dir, "CORDEX_grids.json"),
        CORDEX_domain="EUR-11",
        time_units=None,
        allow_units_convert=True,
    )
    return filename


def test_cmorizer_subdaily(table):
    ds = cx.tutorial.open_dataset("remo_EUR-11_TEMP2_1hr")
    filename = cmor.cmorize_variable(
        ds,
        "tas",
        mapping_table={"tas": {"varname": "TEMP2"}},
        cmor_table=os.path.join(table_dir, f"CORDEX_{table}.json"),
        dataset_table=os.path.join(table_dir, "CORDEX_remo_example.json"),
        grids_table=os.path.join(table_dir, "CORDEX_grids.json"),
        CORDEX_domain="EUR-11",
        time_units=None,
        allow_units_convert=True,
        allow_resample=True,
    )
    return filename

## Example from `fx` table

In [5]:
f = test_cmorizer_fx()

/Users/buntemey/python/packages/py-cordex/cordex/cmor/utils.py:334: UserWarning: writing temporary table to /var/folders/xf/4_4br5p151d003jrl3qnb3gh0000gp/T/tmpuwygjev_
  warn(f"writing temporary table to {filename}")


Let's have a look at the filename.

In [6]:
f

'CORDEX/CORDEX/CMIP6/RCM/EUR-11/GERICS/ECMWF-ERA5/evaluation/r1i1p1f1/REMO2020/v1/fx/orog/v20231023/orog_EUR-11_ECMWF-ERA5_evaluation_r1i1p1f1_GERICS_REMO2020_v1_fx.nc'

In [7]:
!ncdump -h $f

netcdf orog_EUR-11_ECMWF-ERA5_evaluation_r1i1p1f1_GERICS_REMO2020_v1_fx {
dimensions:
	rlat = 412 ;
	rlon = 424 ;
	bnds = 2 ;
	vertices = 4 ;
variables:
	double rlat(rlat) ;
		rlat:units = "degrees" ;
		rlat:axis = "Y" ;
		rlat:long_name = "latitude in rotated pole grid" ;
		rlat:standard_name = "grid_latitude" ;
	double rlon(rlon) ;
		rlon:units = "degrees" ;
		rlon:axis = "X" ;
		rlon:long_name = "longitude in rotated pole grid" ;
		rlon:standard_name = "grid_longitude" ;
	int rotated_latitude_longitude ;
		rotated_latitude_longitude:grid_mapping_name = "rotated_latitude_longitude" ;
		rotated_latitude_longitude:grid_north_pole_latitude = 39.25 ;
		rotated_latitude_longitude:grid_north_pole_longitude = -162. ;
		rotated_latitude_longitude:north_pole_grid_longitude = 0. ;
	double latitude(rlat, rlon) ;
		latitude:standard_name = "latitude" ;
		latitude:long_name = "latitude" ;
		latitude:units = "degrees_north" ;
		latitude:missing_value = 1.e+20 ;
		latitude:_FillValue = 1.e+20 ;
		l

The xarray dataset representation allows to explore the dataset interactively.

In [ ]:
ds = xr.open_dataset(f)
ds

In [ ]:
ds.orog.plot()

In [ ]:
!cdo verifygrid $f

In [ ]:
!cfchecks $f

In [ ]:
!compliance-checker --test=cf:1.7 $f

## Example from `mon` table

In [ ]:
f = test_cmorizer_mon()
f

In [ ]:
ds = xr.open_dataset(f)
ds

In [ ]:
ds.cf

In [ ]:
ds.tas.plot(col="time", col_wrap=4)

In [ ]:
!ncdump -h $f

In [ ]:
!cfchecks $f

In [ ]:
!compliance-checker --test=cf:1.7 $f

## Example from daily and subdaily tables

In [ ]:
f = test_cmorizer_subdaily("1hr")
f

In [ ]:
ds = xr.open_dataset(f)
ds

In [ ]:
!ncdump -h $f

In [ ]:
!cfchecks $f

In [ ]:
!compliance-checker --test=cf:1.7 $f

In [ ]:
# cordex.cmor will automatically resample hourly to other frequencies
# depending on the frequency in the cmor table and the frequency in the
# input dataset
f = test_cmorizer_subdaily("day")
f

In [ ]:
ds = xr.open_dataset(f)
ds

In [ ]:
ds.tas.plot(col="time")

In [ ]:
!ncdump -h $f

In [ ]:
!cfchecks $f

In [ ]:
!compliance-checker --test=cf:1.7 $f